## Loading library and making instance off class


In [1]:
import datetime as dt
from dateutil import parser
import plotly.graph_objects as go
import pandas as pd
import sys
sys.path.append("../")

In [2]:
from api.oanda_api import OandaApi
from infrastructure.instrument_collection import InstrumentCollection as ic
from db.db import DataBaseMongo
pd.set_option('display.max_rows', None)

In [3]:
database = DataBaseMongo()
icol = ic()
api = OandaApi()

In [4]:
load_icol = icol.DB_LoadInstruments()

## Preparing the pairs


In [5]:
pairs = []
for pair, val in icol.instruments_dict.items():
    if val.ins_type == "CURRENCY" and "GBP" in pair:
        pairs.append(pair)

In [6]:
pairs[0:5]

['EUR_GBP', 'GBP_NZD', 'GBP_PLN', 'GBP_CHF', 'GBP_CAD']

In [7]:
candle_data = []
for pair in pairs:
    print(pair)
    candle_temp = api.get_candles_df(
        pair_name=pair, granularity='D', count=3000)
    candle_temp['gain'] = (
        (candle_temp.mid_c - candle_temp.mid_o) / candle_temp.mid_o) * 100
    candle_temp['pair'] = pair
    if '_GBP' in pair:
        candle_temp['gain'] = candle_temp['gain'] * -1
    candle_data.append(candle_temp)
candles_df = pd.concat(candle_data)
candles_df.reset_index(drop=True, inplace=True)
candles_df.time = candles_df.time.dt.date

EUR_GBP
GBP_NZD
GBP_PLN
GBP_CHF
GBP_CAD
GBP_HKD
GBP_ZAR
GBP_JPY
GBP_AUD
GBP_SGD
GBP_USD


## Querying for data from database


In [8]:
forex_calendar_data = database.query_all(DataBaseMongo.FOREX_CALENDAR)
forex_calendar_df = pd.DataFrame.from_dict(forex_calendar_data)
forex_calendar_df['date'] = pd.to_datetime(
    forex_calendar_df['date']).dt.date
forex_calendar_df_uk = forex_calendar_df[forex_calendar_df.country == 'united kingdom'].copy(
)

## Doing some quick cleaning off data


In [9]:
for column in ['actual', 'previous', 'forecast']:
    for symbol in ['$', 'B', 'K', '£', '¥', '€', '₩', '%']:
        forex_calendar_df_uk[column] = forex_calendar_df_uk[column].str.replace(
            symbol, '')
    forex_calendar_df_uk[column] = forex_calendar_df_uk[column].replace(
        '', '0')
    forex_calendar_df_uk[column] = forex_calendar_df_uk[column].astype(float)

## Matching date and preparing for merge


In [10]:
date_match = {}
for original_date in forex_calendar_df_uk.date.unique():
    matched_date = original_date
    tries = 0
    while matched_date not in candles_df.time.values:
        matched_date = matched_date + dt.timedelta(days=1)
        tries += 1
        if tries > 5:
            print("Failed to find a matching date.")
            break
    date_match[original_date] = matched_date

In [11]:
forex_calendar_df_uk['original_date'] = forex_calendar_df_uk['date']
forex_calendar_df_uk['date'] = [date_match[original_date]
                                for original_date in forex_calendar_df_uk['date']]
forex_calendar_df_uk['delta_previous'] = forex_calendar_df_uk['actual'] - \
    forex_calendar_df_uk['previous']
forex_calendar_df_uk['delta_forecast'] = forex_calendar_df_uk['actual'] - \
    forex_calendar_df_uk['forecast']

## Merging the data


In [12]:
candles_analyze = candles_df[['time', 'pair', 'gain']].copy()

In [13]:
merged_dataframe = pd.merge(

    left=candles_analyze, right=forex_calendar_df_uk, left_on='time', right_on='date')

In [17]:
merged_dataframe.category.unique()

array(['balance of trade', 'inflation rate', 'claimant count change',
       'consumer confidence', 'calendar', 'interest rate',
       'retail sales mom', 'composite pmi', 'monthly gdp mom',
       'monthly gdp yoy', 'manufacturing pmi', 'services pmi',
       'unemployment rate', 'gdp growth rate', 'gdp annual growth rate'],
      dtype=object)

In [34]:
category_select1 = 'manufacturing pmi'
category_select2 = 'interest rate'
category_select3 = 'claimant count change'
category_select4 = 'consumer confidence'
category_select5 = 'calendar'
category_select6 = 'balance of trade'
category_select7 = 'retail sales mom'
category_select8 = 'composite pmi'
category_select9 = 'monthly gdp mom'
category_select10 = 'monthly gdp yoy'
category_select11 = 'services pmi'
category_select12 = 'unemployment rate'
category_select13 = 'gdp growth rate'
category_select14 = 'inflation rate'
category_select15 = 'gdp annual growth rate'

merged_catagory_dataframe = merged_dataframe[merged_dataframe.category ==
                                             category_select2]

In [35]:
merged_catagory_dataframe[0:5]

,time,pair,gain,date,country,category,event,symbol,actual,previous,forecast,original_date,delta_previous,delta_forecast
9,2020-03-11,EUR_GBP,-1.231830,2020-03-11,united kingdom,interest rate,boe press conference,,0.0,0.0,0.0,2020-03-11,0.0,0.0
110,2021-12-16,EUR_GBP,0.198704,2021-12-16,united kingdom,interest rate,boe quantitative easing,,875.0,875.0,875.0,2021-12-16,0.0,0.0
156,2022-06-29,EUR_GBP,0.070813,2022-06-29,united kingdom,interest rate,boe gov bailey speech,,0.0,0.0,0.0,2022-06-29,0.0,0.0
177,2022-09-27,EUR_GBP,-0.069385,2022-09-27,united kingdom,interest rate,boe pill speech,,0.0,0.0,0.0,2022-09-27,0.0,0.0
194,2022-12-15,EUR_GBP,0.042400,2022-12-15,united kingdom,interest rate,boe interest rate decision,UKBRBASE,3.5,3.0,3.5,2022-12-15,0.5,0.0


## Analyzing the data


In [47]:
CATEGORY_CHECKER = category_select6
dataframe_analyzing = merged_dataframe[merged_dataframe.category ==
                                       CATEGORY_CHECKER]

print(f"Checking {CATEGORY_CHECKER}...")
print(f"Total gain when delta_previous < 0: {
      dataframe_analyzing[dataframe_analyzing.delta_previous < 0].gain.sum()}")
print(f"Total gain when delta_previous <= 0: {
      dataframe_analyzing[dataframe_analyzing.delta_previous <= 0].gain.sum()}")
print(f"Total gain when delta_previous >= 0: {
      dataframe_analyzing[dataframe_analyzing.delta_previous >= 0].gain.sum()}")
print(f"Total gain when delta_previous > 0: {
      dataframe_analyzing[dataframe_analyzing.delta_previous > 0].gain.sum()}")
print(f"Total gain when delta_forecast < 0: {
      dataframe_analyzing[dataframe_analyzing.delta_forecast < 0].gain.sum()}")
print(f"Total gain when delta_forecast <= 0: {
      dataframe_analyzing[dataframe_analyzing.delta_forecast <= 0].gain.sum()}")
print(f"Total gain when delta_forecast >= 0: {
      dataframe_analyzing[dataframe_analyzing.delta_forecast >= 0].gain.sum()}")
print(f"Total gain when delta_forecast > 0: {
      dataframe_analyzing[dataframe_analyzing.delta_forecast > 0].gain.sum()}")

Checking balance of trade...
Total gain when delta_previous < 0: -4.460628258466712
Total gain when delta_previous <= 0: -4.460628258466712
Total gain when delta_previous >= 0: 4.60997356574532
Total gain when delta_previous > 0: 4.60997356574532
Total gain when delta_forecast < 0: -4.637054437407745
Total gain when delta_forecast <= 0: -4.637054437407745
Total gain when delta_forecast >= 0: 4.786399744686352
Total gain when delta_forecast > 0: 4.786399744686352


-4.47878095610241